In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [40]:
data = pd.read_csv("./weatherAUS.csv")

data['Date'] = pd.to_datetime(data['Date']).dt.month
data = data.rename({'Date': 'Month'}, axis=1)

binary = ['RainToday', 'RainTomorrow']
for column in binary:
    data[column] = data[column].map({'Yes': 1, 'No': 0})

categorical = ['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm']
for column in categorical:
    data[column] = data[column].astype('category').cat.codes.replace(-1, np.nan)

In [41]:
target = data.pop('RainTomorrow').iloc[:-1]

#Замена NaN на усредненноые по месяцам значения
for column in data.columns.values[np.isnan(data).max()]:
    monthly_average_for_index = data['Month'].map(data.groupby('Month')[column].mean())
    data[column] = data[column].fillna(monthly_average_for_index)

#Удаление NaN из целевой переменной
unknown, data = data.iloc[-1], data.iloc[:-1]
data = data.loc[pd.notnull(target)]
target = target.dropna()

In [42]:
data = data.reset_index(drop=True)
target = target.reset_index(drop=True)

In [43]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.25, shuffle=False)

## Наивный Байес

In [44]:
from sklearn.naive_bayes import BernoulliNB

classifier = BernoulliNB()
classifier.fit(X_train, y_train)
prediction = classifier.predict(X_test)

(prediction == y_test).mean()

0.7793749472559003

In [45]:
from sklearn.naive_bayes import GaussianNB

classifier = GaussianNB()
classifier.fit(X_train, y_train)
prediction = classifier.predict(X_test)

(prediction == y_test).mean()

0.8325972601198346

In [46]:
#Логарифмирование некоторых признаков
def logfunc(df, indices):
    res = df.copy()
    for i in indices:
        res.iloc[:,i] = np.log1p(res.iloc[:,i])
    return res

In [47]:
%%time

ind = [5,8,11]
classifier = GaussianNB()
classifier.fit(logfunc(X_train, ind), y_train)
prediction = classifier.predict(logfunc(X_test, ind))

(prediction == y_test).mean()

Wall time: 170 ms


0.8332161242229036

In [48]:
from sklearn import metrics
all_metrics = pd.DataFrame(index=['Naive Bayes', 'Nearest Neighbors', 'Logit Regression'],
                      columns=['Accuracy', 'Precision', 'Recall', 'F1'])

In [49]:
all_metrics.iloc[0, 0] = metrics.accuracy_score(y_test, prediction)
all_metrics.iloc[0, 1] = metrics.precision_score(y_test, prediction)
all_metrics.iloc[0, 2] = metrics.recall_score(y_test, prediction)
all_metrics.iloc[0, 3] = metrics.f1_score(y_test, prediction)

## Логистическая регрессия

In [50]:
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm

In [51]:
classifier = LogisticRegression(penalty='l2', max_iter=5000)
classifier.fit(X_train, y_train)
prediction = classifier.predict(X_test)

(prediction == y_test).mean()

0.8440462460266112

In [52]:
model = sm.Logit(y_train, X_train)
result = model.fit(maxiter=1000, disp=0)
print (np.mean((result.predict(X_test) >= 0.5).astype(int).values == y_test))

0.8439899856536048


In [53]:
#Добавим столбец единиц
X_train_ones = X_train.join(pd.DataFrame({'ones': np.ones(len(X_train))}))
X_test_ones = X_test.join(pd.DataFrame({'ones': np.ones(len(data))}))

In [54]:
%%time

result = sm.Logit(y_train, X_train_ones).fit(maxiter=1000, disp=0)
print (np.mean((result.predict(X_test_ones) >= 0.5).astype(int).values == y_test))

0.8473374778474781
Wall time: 655 ms


#### Отбор признаков

In [55]:
from sys import float_info

def forward_selected(data, response):
    remaining = set(data.columns)
    selected = []
    current_score, best_new_score = float_info.max, float_info.max
    while remaining and current_score == best_new_score:
        scores_with_candidates = []
        for candidate in remaining:
            score = sm.Logit(response, data[selected + [candidate]]).fit(maxiter=1000, disp=0).bic
            scores_with_candidates.append((score, candidate))
        scores_with_candidates.sort(reverse=True)
        best_new_score, best_candidate = scores_with_candidates.pop()
        if current_score > best_new_score:
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score
    model = sm.Logit(response, data[selected]).fit(maxiter=1000, disp=0)
    return model, selected

In [58]:
result = forward_selected(X_train_ones, y_train)

prediction = (result[0].predict(X_test_ones[result[1]]) > .5).astype('int').values
(prediction == y_test).mean()

0.8475062589664969

In [59]:
all_metrics.iloc[2, 0] = metrics.accuracy_score(y_test, prediction)
all_metrics.iloc[2, 1] = metrics.precision_score(y_test, prediction)
all_metrics.iloc[2, 2] = metrics.recall_score(y_test, prediction)
all_metrics.iloc[2, 3] = metrics.f1_score(y_test, prediction)

In [60]:
result[1].remove('ones')
selected = result[1]
selected

['Sunshine',
 'Humidity3pm',
 'Pressure3pm',
 'WindGustSpeed',
 'RainToday',
 'Pressure9am',
 'Cloud3pm',
 'WindSpeed3pm',
 'Temp3pm',
 'Location',
 'Rainfall',
 'WindSpeed9am',
 'MaxTemp',
 'WindDir9am',
 'Month',
 'WindGustDir']

## Ближайшие соседи

In [61]:
from sklearn.neighbors import KNeighborsClassifier

In [62]:
#Удаление коррелирующих элементов
corr_mtrx = np.corrcoef(X_train[selected].T)
corr_ind = np.where(corr_mtrx > .95)
for i in range(len(selected)):
    if corr_ind[0][i] != corr_ind[1][i]:
        selected.pop(corr_ind[0][i])

In [63]:
%%time

neigh = KNeighborsClassifier(n_neighbors=20)
neigh.fit(X_train[selected], y_train)

prediction = neigh.predict(X_test[selected])
(prediction == y_test).mean()

Wall time: 28.6 s


0.8437649441615798

In [64]:
all_metrics.iloc[1, 0] = metrics.accuracy_score(y_test, prediction)
all_metrics.iloc[1, 1] = metrics.precision_score(y_test, prediction)
all_metrics.iloc[1, 2] = metrics.recall_score(y_test, prediction)
all_metrics.iloc[1, 3] = metrics.f1_score(y_test, prediction)

## Итог

В ходе задания были построены три модели для решения задачи классификации.

Была проведена некоторая работа с данными, в частности, все данные были приведены к числовым, отсутсвующие данные были дополнены средними значениями и т.д. Также был осуществлен отбор признаков, что улучшило точность и время работы методов.

Были получены следующие метрики:

In [69]:
all_metrics.astype('float').round(3)

,Accuracy,Precision,Recall,F1
Naive Bayes,0.833,0.626,0.563,0.593
Nearest Neighbors,0.844,0.778,0.386,0.516
Logit Regression,0.848,0.748,0.441,0.555


Поскольку классы в выборке несбалансированы, accuracy получился больше остальных метрик, но именно они точнее говорят о результатах. Если ориентироваться на F-меру, то лучший результат получился с помощью Байесовского классификатора (распределение Гаусса), он же был и самым быстрым. Наименее точным, и при этом самым ресурсоемким оказался метод ближайших соседей.